In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [147]:
# Load data
path = '/home/han/repos/BEACO2N-CO2-Correction/data/'
X_train  = torch.Tensor(np.load(path + 'X_train.npy', allow_pickle=True))
X_test   = torch.Tensor(np.load(path + 'X_test.npy', allow_pickle=True))
X_val    = torch.Tensor(np.load(path + 'X_val.npy', allow_pickle=True))
y_train  = torch.Tensor(np.load(path + 'y_train.npy', allow_pickle=True))
y_test   = torch.Tensor(np.load(path + 'y_test.npy', allow_pickle=True))
y_val    = torch.Tensor(np.load(path + 'y_val.npy', allow_pickle=True))

In [161]:
# Parameters
n_iters      = 50
nsize_in     = X_train.shape[1] 
nsize_out    = 1
nsize_hidden = X_train.shape[1]
n_layers     = 4
activations = [nn.Sigmoid(),#nn.ReLU(),
               nn.Sigmoid(),#nn.ReLU(),
               nn.Sigmoid(),#nn.ReLU(), 
               None]
layer_combiners = [nn.Linear(nsize_in, nsize_hidden),
                   nn.Linear(nsize_in, nsize_hidden-1),
                   nn.Linear(nsize_hidden-1, nsize_hidden-2),
                   nn.Linear(nsize_hidden-2, nsize_out)]
learning_rate = 0.1

In [162]:
def create_model(n_layers, activation, combiners):
    
    if len(activation) != n_layers:
        act = activation[0]*(n_layers - 1)
        act.append(activation[-1])
    else:
        act = activations
    if len(combiners) != n_layers:
        combine = combiners[0]*(n_layers - 1)
        combine.append(combiners[-1])
    else:
        combine = combiners
        
    for layer in range(n_layers):
        if layer == 0:
            model = nn.Sequential(combine[layer])
            if act[layer]:
                model.add_module(name=str(2*layer+1), module=act[layer])
        else:
            model.add_module(name=str(2*layer),module=combine[layer])
            if act[layer]:
                model.add_module(name=str(2*layer+1), module=act[layer])
                
    return model

In [163]:
try:
    del model
except:
    pass
model = create_model(n_layers, activations, layer_combiners)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [165]:
def run(n_iters, lr=0.1,restart=False, pr=True):
    losses
    if restart:
        try:
            del model
        except:
            pass
        model = create_model(n_layers, activations, layer_combiners)
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(),lr=lr)
    for epoch in range(n_iters):
       # Forward propagation
        y_pred = model(X_train)

        #Compute and print loss
        loss  = criterion(y_pred, y_train)
        losses[epoch] = loss.item()
        if pr:
            print('epoch: ',epoch, ' loss: ', loss.item())
        if torch.isnan(loss):
            break

        #Zero the gradients (so that gradients don't accumulate from epoch to epoch)
        optimizer.zero_grad()

        #perform a backward pass (backpropagation)
        loss.backward()

        # Update the parameters
        optimizer.step()
    return lo

In [86]:
# Lets Evaluate
model.eval()
print(model.training ) #Just to be sure

ypred = model(X_test)
loss  = criterion(y_pred, y_test)
print("Loss: ", loss)

False
Loss:  tensor(443.4062, grad_fn=<MseLossBackward>)


/home/han/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:445: UserWarning: Using a target size (torch.Size([6327])) that is different to the input size (torch.Size([18981, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [156]:
model.eval()
ypred = model(X_test)
err = torch.norm(ypred-y_test, p=2)
print(y_test[1:10])
print(ypred[1:10])
print(err)

tensor([413.0459, 416.9233, 412.2977, 490.5103, 411.9405, 413.0804, 418.6821,
        436.1230, 414.5334])
tensor([[423.5479],
        [423.5479],
        [423.5479],
        [423.5479],
        [423.5479],
        [423.5479],
        [423.5479],
        [423.5479],
        [423.5479]], grad_fn=<SliceBackward>)
tensor(134256.1250, grad_fn=<NormBackward0>)


In [90]:
# Lets search for best learning rate
learning_rates = [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 2, 10, 100]
for lr in learning_rates:
    run(200)


tensor([433.2132, 410.1967, 465.3893, 432.4657, 417.1220, 416.7152, 413.6294,
        413.1808, 420.4663])

In [170]:
loss.item()

415.7703552246094

In [171]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_forward_unimplemented',
 '_get_item_by_idx',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_vers

In [175]:
model.share_memory

<bound method Module.share_memory of Sequential(
  (0): Linear(in_features=7, out_features=7, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=7, out_features=6, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=6, out_features=5, bias=True)
  (5): Sigmoid()
  (6): Linear(in_features=5, out_features=1, bias=True)
)>